In [23]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [24]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

In [25]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [26]:
plots_path = os.path.join(current_dir, 'plots')
tables_path = os.path.join(current_dir, 'tables')

In [27]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

params_names = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
          'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost', 'EnergyRevenue', 'RegRevenue']

param_print_names = params_names

for i in range(len(param_print_names)):
    param_print_names[i] = param_print_names[i].replace("_", " ")

cost_params = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
        'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost']

revenue_params = ['EnergyRevenue', 'RegRevenue', 'RsvRevenue']

In [28]:
# get load data

load_data_raw = pd.read_csv(cem_path + '\\Thermal_Base\\' + 'system\\' + 'Demand_data.csv')
load_data = load_data_raw['Demand_MW_z1']

In [29]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

filename = 'operational_profits_' + date + '.pdf'

In [30]:
unit_profit_dic = {case: None for case in case_names}
percentage_profits_dic = {case: None for case in case_names}
consumer_payments_dic = {case: None for case in case_names}
total_investment_costs_dic = {case: None for case in case_names}
total_operating_costs_dic = {case: None for case in case_names}
total_system_costs_dic = {case: None for case in case_names} 
total_inv_op_costs_dic = {case: None for case in case_names}
total_objective_dic = {case: None for case in case_names}
nse_costs_dic = {case: None for case in case_names}
total_nse_dic = {case: None for case in case_names}
total_unmet_rsv_dic = {case: None for case in case_names}
total_nse_cost_dic = {case: None for case in case_names}
total_unmet_rsv_cost_dic = {case: None for case in case_names}
total_fixed_om_costs_dic = {case: None for case in case_names}
total_var_om_out_costs_dic = {case: None for case in case_names}
total_fuel_costs_dic = {case: None for case in case_names}
total_var_om_in_costs_dic = {case: None for case in case_names}
total_start_costs_dic = {case: None for case in case_names}
total_charge_costs_dic = {case: None for case in case_names}

In [37]:
vre_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Vre.csv')

In [40]:
vre_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Vre.csv')
thermal_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Thermal.csv')
storage_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Storage.csv')

In [41]:
dfGen = pd.concat([vre_dfGen, thermal_dfGen, storage_dfGen], ignore_index=True)

In [42]:
dfGen

,Resource,Zone,Num_VRE_Bins,New_Build,Can_Retire,Existing_Cap_MW,Max_Cap_MW,Min_Cap_MW,Inv_Cost_per_MWyr,Fixed_OM_Cost_per_MWyr,...,Max_Cap_MWh,Min_Cap_MWh,Inv_Cost_per_MWhyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh_In,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration
0,Land-Based Wind - Class 1 - Technology 1,1,1.0,1,0,32791.061172,-1,0,83289.329169,32443.047267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Utility PV - Class 1,1,1.0,1,0,17972.934821,-1,0,74134.164015,23765.603456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NG 2-on-1 Combined Cycle (F-Frame),1,NaN,1,0,46274.023418,-1,0,56221.989154,25875.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NG Combustion Turbine (F-Frame),1,NaN,1,0,9457.159921,-1,0,50580.567617,19575.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
for case_name in case_names:

    print('Case Name: ' + case_name + '\n')

    # load generator characteristics from resources folder
    vre_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Vre.csv')
    thermal_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Thermal.csv')
    storage_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Storage.csv')
    # combine all resources to dfGen
    dfGen = pd.concat([vre_dfGen, thermal_dfGen, storage_dfGen], ignore_index=True)

    gen_capacity_gw = dfGen['Existing_Cap_MW'] / ModelScalingFactor
    gen_capacity_mw = dfGen['Existing_Cap_MW']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh'] / ModelScalingFactor

    resource_list = dfGen['Resource']

    # solar and wind id
    solar_id = dfGen[dfGen['Resource'] == 'Utility PV - Class 1'].index[0]
    wind_id = dfGen[dfGen['Resource'] == 'Land-Based Wind - Class 1 - Technology 1'].index[0]

    cem_case = 'CEM_' + case_name + '_ABB\\'
    pf_case = 'PF_' + case_name + '\\'
    dlac_case = 'DLAC_' + case_name + '\\'
    dlac_imperfect_case = 'DLAC_imperfect_' + case_name + '\\'
    slac_case = 'SLAC_' + case_name + '\\'

    # define print names
    cem_case_print = cem_case.replace('_', ' ')
    cem_case_print = cem_case_print.replace('\\', '')
    cem_case_print = cem_case_print.replace('ABB', '')

    pf_case_print = pf_case.replace('_', ' ')
    pf_case_print = pf_case_print.replace('\\', '')

    dlac_case_print = dlac_case.replace('_', ' ')
    dlac_case_print = dlac_case_print.replace('\\', '')

    dlac_imperfect_case_print = dlac_imperfect_case.replace('_', ' ')
    dlac_imperfect_case_print = dlac_imperfect_case_print.replace('\\', '')

    slac_case_print = slac_case.replace('_', ' ')
    slac_case_print = slac_case_print.replace('\\', '')

    cem_results_path = cem_path + '\\' + case_name + '\\' + 'results\\'
    pf_results_path = policies_path + '\\' + case_name + '\\' + 'results_pf\\'
    dlac_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-p\\'
    dlac_imperfect_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-i\\'
    slac_results_path = policies_path + '\\' + case_name + '\\' + 'results_slac\\'

    cem_raw = pd.read_csv(cem_results_path + 'NetRevenue.csv')
    pf_raw = pd.read_csv(pf_results_path + 'NetRevenue.csv')
    dlac_raw = pd.read_csv(dlac_results_path + 'NetRevenue.csv')
    dlac_imperfect_raw = pd.read_csv(dlac_imperfect_results_path + 'NetRevenue.csv')
    slac_raw = pd.read_csv(slac_results_path + 'NetRevenue.csv')

    ### XXX Fix inv cost MWh
    # dlac_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']
    # dlac_imperfect_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']
    # slac_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']


    cem_raw_prices = pd.read_csv(cem_results_path + 'prices.csv')
    pf_raw_prices = pd.read_csv(pf_results_path + 'price_electricity.csv', header=None)
    dlac_raw_prices = pd.read_csv(dlac_results_path + 'price_electricity.csv', header=None)
    dlac_imperfect_raw_prices = pd.read_csv(dlac_imperfect_results_path + 'price_electricity.csv', header=None)
    slac_raw_prices = pd.read_csv(slac_results_path + 'price_electricity.csv', header=None)


    column_names = cem_raw.columns[1:]
    resource_names = cem_raw['Resource'].unique()
    gen_print_names = resource_names
    print(gen_print_names)

    param_print_names = params_names

    for i in range(len(param_print_names)):
        param_print_names[i] = param_print_names[i].replace("_", " ")

    for i in range(len(resource_names)):
        gen_print_names[i] = gen_print_names[i].replace("Coal-new", "Coal")
        gen_print_names[i] = gen_print_names[i].replace("NG Combustion Turbine (F-Frame)", "NG CT")
        gen_print_names[i] = gen_print_names[i].replace("NG Combined Cycle (F-Frame)", "NG CC")
        gen_print_names[i] = gen_print_names[i].replace("Utility PV - Class 1", "Solar")
        gen_print_names[i] = gen_print_names[i].replace("Land-Based Wind - Class 1 - Technology 1", "Wind")
        gen_print_names[i] = gen_print_names[i].replace("Utility-Scale Battery Storage", "Battery")

    # get cem total nse and unmet rsv and total cem nse and unmet rsv costs
    cem_costs_genx = pd.read_csv(cem_results_path + 'costs.csv')
    cem_sum_ur_cost = cem_costs_genx[cem_costs_genx['Costs'] == 'cUnmetRsv']['Total'].values[0]
    cem_total_unmet_rsv = cem_sum_ur_cost / 1000
    cem_total_nse_cost = cem_costs_genx[cem_costs_genx['Costs'] == 'cNSE']['Total'].values[0]
    cem_total_nse = cem_total_nse_cost / 5000

    # load in timeseries of nse and unmet rsv from LACs
    cem_nse = pd.read_csv(cem_results_path + 'nse.csv')
    pf_nse = pd.read_csv(pf_results_path + 'zone_nse.csv', header=None)
    dlac_nse = pd.read_csv(dlac_results_path + 'zone_nse.csv', header=None)
    dlac_imperfect_nse = pd.read_csv(dlac_imperfect_results_path + 'zone_nse.csv', header=None)
    slac_nse = pd.read_csv(slac_results_path + 'zone_nse.csv', header=None)

    dlac_ur = pd.read_csv(dlac_results_path + 'zone_unmet_rsv.csv', header=None)
    pf_ur = pd.read_csv(pf_results_path + 'zone_unmet_rsv.csv', header=None)
    dlac_imperfect_ur = pd.read_csv(dlac_imperfect_results_path + 'zone_unmet_rsv.csv', header=None)
    slac_ur = pd.read_csv(slac_results_path + 'zone_unmet_rsv.csv', header=None)

    pf_nse_total = pf_nse.sum(axis=1)
    dlac_nse_total = dlac_nse.sum(axis=1)
    dlac_imperfect_nse_total = dlac_imperfect_nse.sum(axis=1)
    slac_nse_total = slac_nse.sum(axis=1)

    pf_ur_total = pf_ur.sum(axis=1)
    dlac_ur_total = dlac_ur.sum(axis=1)
    dlac_imperfect_ur_total = dlac_imperfect_ur.sum(axis=1)
    slac_ur_total = slac_ur.sum(axis=1)

    # print(dlac_nse_total)

    nse_totals = {
        'CEM': f"{cem_total_nse:.4e}",
        'PF': f"{pf_nse_total[0]:.4e}",
        'DLAC': f"{dlac_nse_total[0]:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_nse_total[0]:.4e}",
        'SLAC': f"{slac_nse_total[0]:.4e}"
    }

    unmet_rsv_totals = {
        'CEM': f"{cem_total_unmet_rsv:.4e}",
        'PF': f"{pf_ur_total[0]:.4e}",
        'DLAC': f"{dlac_ur_total[0]:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_ur_total[0]:.4e}",
        'SLAC': f"{slac_ur_total[0]:.4e}"
    }


    total_nse_dic[case_name] = nse_totals
    total_unmet_rsv_dic[case_name] = unmet_rsv_totals

    # calculate reg and rsv revenue sums
    cem_reg_revenue_sum = cem_raw['RegRevenue'].sum()
    cem_rsv_revenue_sum = cem_raw['RsvRevenue'].sum()

    pf_reg_revenue_sum = pf_raw['RegRevenue'].sum()
    pf_rsv_revenue_sum = pf_raw['RsvRevenue'].sum()

    dlac_reg_revenue_sum = dlac_raw['RegRevenue'].sum()
    dlac_rsv_revenue_sum = dlac_raw['RsvRevenue'].sum()

    dlac_imperfect_reg_revenue_sum = dlac_imperfect_raw['RegRevenue'].sum()
    dlac_imperfect_rsv_revenue_sum = dlac_imperfect_raw['RsvRevenue'].sum()

    slac_reg_revenue_sum = slac_raw['RegRevenue'].sum()
    slac_rsv_revenue_sum = slac_raw['RsvRevenue'].sum()



    # load unmet reserves cost information
    pf_ur_cost = pd.read_csv(pf_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_ur_cost = pd.read_csv(dlac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_imperfect_ur_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    slac_ur_cost = pd.read_csv(slac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)

    pf_nse_cost = pd.read_csv(pf_results_path + 'revenue_nse_cost.csv', header=None)
    dlac_nse_cost = pd.read_csv(dlac_results_path + 'revenue_nse_cost.csv', header=None)
    dlac_imperfect_nse_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_nse_cost.csv', header=None)
    slac_nse_cost = pd.read_csv(slac_results_path + 'revenue_nse_cost.csv', header=None)

    pf_sum_nse_cost = pf_nse_cost.sum(axis=1)[0]
    dlac_sum_nse_cost = dlac_nse_cost.sum(axis=1)[0]
    dlac_imperfect_sum_nse_cost = dlac_imperfect_nse_cost.sum(axis=1)[0]
    slac_sum_nse_cost = slac_nse_cost.sum(axis=1)[0]

    pf_sum_ur_cost = pf_ur_cost.sum(axis=1)[0]
    dlac_sum_ur_cost = dlac_ur_cost.sum(axis=1)[0]
    dlac_imperfect_sum_ur_cost = dlac_imperfect_ur_cost.sum(axis=1)[0]
    slac_sum_ur_cost = slac_ur_cost.sum(axis=1)[0]

    total_nse_cost_dic[case_name] = {
        'CEM': f"{cem_total_nse_cost:.4e}",
        'PF': f"{pf_sum_nse_cost:.4e}",
        'DLAC': f"{dlac_sum_nse_cost:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_sum_nse_cost:.4e}",
        'SLAC': f"{slac_sum_nse_cost:.4e}"
    }

    total_unmet_rsv_cost_dic[case_name] = {
        'CEM': f"{cem_sum_ur_cost:.4e}",
        'PF': f"{pf_sum_ur_cost:.4e}",
        'DLAC': f"{dlac_sum_ur_cost:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_sum_ur_cost:.4e}",
        'SLAC': f"{slac_sum_ur_cost:.2e}"
    }

    cem_prices = cem_raw_prices['1']
    pf_prices = pf_raw_prices[0]
    dlac_prices = dlac_raw_prices[0]
    dlac_imperfect_prices = dlac_imperfect_raw_prices[0]
    slac_prices = slac_raw_prices[0]

    cem_costs = cem_raw[cost_params]
    pf_costs = pf_raw[cost_params]
    dlac_costs = dlac_raw[cost_params]
    dlac_imperfect_costs = dlac_imperfect_raw[cost_params]
    slac_costs = slac_raw[cost_params]

    cem_economics = cem_raw[cost_params + revenue_params]
    pf_economics = pf_raw[cost_params + revenue_params]
    dlac_economics = dlac_raw[cost_params + revenue_params]
    dlac_imperfect_economics = dlac_imperfect_raw[cost_params + revenue_params]
    slac_economics = slac_raw[cost_params + revenue_params]

    cem_economics_copy = cem_economics.copy()
    pf_economics_copy = pf_economics.copy()
    dlac_economics_copy = dlac_economics.copy()
    dlac_imperfect_economics_copy = dlac_imperfect_economics.copy()
    slac_economics_copy = slac_economics.copy()

    cem_economics_copy[cost_params] = cem_economics_copy[cost_params] * -1
    pf_economics_copy[cost_params] = pf_economics_copy[cost_params] * -1
    dlac_economics_copy[cost_params] = dlac_economics_copy[cost_params] * -1
    dlac_imperfect_economics_copy[cost_params] = dlac_imperfect_economics_copy[cost_params] * -1
    slac_economics_copy[cost_params] = slac_economics_copy[cost_params] * -1

    cem_profit = cem_economics_copy.sum(axis=1)
    pf_profit = pf_economics_copy.sum(axis=1)
    dlac_profit = dlac_economics_copy.sum(axis=1)
    dlac_imperfect_profit = dlac_imperfect_economics_copy.sum(axis=1)
    slac_profit = slac_economics_copy.sum(axis=1)

    cem_economics_scaled = cem_economics_copy / ModelScalingFactor**2
    pf_economics_scaled = pf_economics_copy / ModelScalingFactor**2
    dlac_economics_scaled = dlac_economics_copy / ModelScalingFactor**2
    dlac_imperfect_economics_scaled = dlac_imperfect_economics_copy / ModelScalingFactor**2
    slac_economics_scaled = slac_economics_copy / ModelScalingFactor**2

    cem_scaled_profit = cem_economics_scaled.sum(axis=1)
    pf_scaled_profit = pf_economics_scaled.sum(axis=1)
    dlac_scaled_profit = dlac_economics_scaled.sum(axis=1)
    dlac_imperfect_scaled_profit = dlac_imperfect_economics_scaled.sum(axis=1)
    slac_scaled_profit = slac_economics_scaled.sum(axis=1)

    ###================================================================================================
    ### Total Investment Costs
    ###================================================================================================
    cem_mw_investment_costs = cem_raw['Inv_cost_MW'].sum()
    cem_mwhour_investment_costs = cem_raw['Inv_cost_MWh'].sum()

    pf_mw_investment_costs = pf_raw['Inv_cost_MW'].sum()
    pf_mwhour_investment_costs = pf_raw['Inv_cost_MWh'].sum()

    dlac_mw_investment_costs = dlac_raw['Inv_cost_MW'].sum()
    dlac_mwhour_investment_costs = dlac_raw['Inv_cost_MWh'].sum()

    dlac_imperfect_mw_investment_costs = dlac_imperfect_raw['Inv_cost_MW'].sum()
    dlac_imperfect_mwhour_investment_costs = dlac_imperfect_raw['Inv_cost_MWh'].sum()

    slac_mw_investment_costs = slac_raw['Inv_cost_MW'].sum()
    slac_mwhour_investment_costs = slac_raw['Inv_cost_MWh'].sum()

    cem_total_investment_costs = cem_mw_investment_costs + cem_mwhour_investment_costs
    pf_total_investment_costs = pf_mw_investment_costs + pf_mwhour_investment_costs
    dlac_total_investment_costs = dlac_mw_investment_costs + dlac_mwhour_investment_costs
    dlac_imperfect_total_investment_costs = dlac_imperfect_mw_investment_costs + dlac_imperfect_mwhour_investment_costs
    slac_total_investment_costs = slac_mw_investment_costs + slac_mwhour_investment_costs

    cem_total_investment_costs_sci = f"{cem_total_investment_costs:.4e}"
    pf_total_investment_costs_sci = f"{pf_total_investment_costs:.4e}"
    dlac_total_investment_costs_sci = f"{dlac_total_investment_costs:.4e}"
    dlac_imperfect_total_investment_costs_sci = f"{dlac_imperfect_total_investment_costs:.4e}"
    slac_total_investment_costs_sci = f"{slac_total_investment_costs:.4e}"

    total_investment_costs = {
        'CEM': cem_total_investment_costs_sci,
        'PF': pf_total_investment_costs_sci,
        'DLAC': dlac_total_investment_costs_sci,
        'DLAC Imperfect': dlac_imperfect_total_investment_costs_sci,
        'SLAC': slac_total_investment_costs_sci
    }
    total_investment_costs_dic[case_name] = total_investment_costs


    ###================================================================================================
    ### Unit Profits
    ###================================================================================================

    cem_unit_profit_mw = cem_profit / gen_capacity_mw
    pf_unit_profit_mw = pf_profit / gen_capacity_mw
    dlac_unit_profit_mw = dlac_profit / gen_capacity_mw
    dlac_imperfect_unit_profit_mw = dlac_imperfect_profit / gen_capacity_mw
    slac_unit_profit_mw = slac_profit / gen_capacity_mw

    cem_unit_profit_mw_sci = cem_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    pf_unit_profit_mw_sci = pf_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_unit_profit_mw_sci = dlac_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_imperfect_unit_profit_mw_sci = dlac_imperfect_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    slac_unit_profit_mw_sci = slac_unit_profit_mw.apply(lambda x: f"{x:.4e}")

    cem_unit_profit_gw = cem_profit / gen_capacity_gw
    pf_unit_profit_gw = pf_profit / gen_capacity_gw
    dlac_unit_profit_gw = dlac_profit / gen_capacity_gw
    dlac_imperfect_unit_profit_gw = dlac_imperfect_profit / gen_capacity_gw
    slac_unit_profit_gw = slac_profit / gen_capacity_gw

    unit_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_unit_profit_mw_sci,
        'PF': pf_unit_profit_mw_sci,
        'DLAC': dlac_unit_profit_mw_sci,
        'DLAC Imperfect': dlac_imperfect_unit_profit_mw_sci,
        'SLAC': slac_unit_profit_mw_sci
    })

    # sorted_unit_profits = sorted(unit_profits.items(), key=lambda item: item[1], reverse=True)

    unit_profit_dic[case_name] = unit_profit_df

    ###================================================================================================
    ### Calculate consumer payments
    ###================================================================================================

    cem_consumer_payments = cem_prices * load_data
    pf_consumer_payments = pf_prices * load_data
    dlac_consumer_payments = dlac_prices * load_data
    dlac_imperfect_consumer_payments = dlac_imperfect_prices * load_data 
    slac_consumer_payments = slac_prices * load_data

    sum_cem_cp = cem_consumer_payments.sum() + cem_reg_revenue_sum + cem_rsv_revenue_sum
    sum_pf_cp = pf_consumer_payments.sum() + pf_reg_revenue_sum + pf_rsv_revenue_sum
    sum_dlac_cp = dlac_consumer_payments.sum() + dlac_reg_revenue_sum + dlac_rsv_revenue_sum
    sum_dlac_imperfect_cp = dlac_imperfect_consumer_payments.sum() + \
                                    dlac_imperfect_reg_revenue_sum + dlac_imperfect_rsv_revenue_sum
    sum_slac_cp = slac_consumer_payments.sum()  + slac_reg_revenue_sum + slac_rsv_revenue_sum

    # # Convert sums to millions of dollars
    # sum_cem_cp_mil = sum_cem_cp / 1e6
    # sum_dlac_cp_mil = sum_dlac_cp / 1e6
    # sum_dlac_imperfect_cp_mil = sum_dlac_imperfect_cp / 1e6
    # sum_slac_cp_mil = sum_slac_cp / 1e6

    sum_cem_cp_sci = f"{sum_cem_cp:.4e}"
    sum_pf_cp_sci = f"{sum_pf_cp:.4e}"
    sum_dlac_cp_sci = f"{sum_dlac_cp:.4e}"
    sum_dlac_imperfect_cp_sci = f"{sum_dlac_imperfect_cp:.4e}"
    sum_slac_cp_sci = f"{sum_slac_cp:.4e}"

    # Create a dictionary with the sums
    consumer_payments = {
        'CEM': sum_cem_cp_sci,
        'PF': sum_pf_cp_sci,
        'DLAC': sum_dlac_cp_sci,
        'DLAC Imperfect': sum_dlac_imperfect_cp_sci,
        'SLAC': sum_slac_cp_sci
    }

    # # Sort the dictionary by values in descending order
    # sorted_consumer_payments = sorted(consumer_payments.items(), key=lambda item: item[1], reverse=True)

    consumer_payments_dic[case_name] = consumer_payments

    ###================================================================================================
    ### Calculate Total System Costs
    ###================================================================================================

    # load unmet reserves cost information
    pf_ur_cost = pd.read_csv(pf_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_ur_cost = pd.read_csv(dlac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_imperfect_ur_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    slac_ur_cost = pd.read_csv(slac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)

    pf_sum_ur_cost = pf_ur_cost.sum(axis=1)[0]
    dlac_sum_ur_cost = dlac_ur_cost.sum(axis=1)[0]
    dlac_imperfect_sum_ur_cost = dlac_imperfect_ur_cost.sum(axis=1)[0]
    slac_sum_ur_cost = slac_ur_cost.sum(axis=1)[0]

    # nse_costs_dic[case_name] = {} # XXX why is this here?

    cem_total_system_costs = cem_costs.sum().sum() + cem_total_nse_cost
    pf_total_system_costs = pf_costs.sum().sum() + pf_sum_nse_cost
    dlac_total_system_costs = dlac_costs.sum().sum() + dlac_sum_nse_cost
    dlac_imperfect_total_system_costs = dlac_imperfect_costs.sum().sum() + dlac_imperfect_sum_nse_cost
    slac_total_system_costs = slac_costs.sum().sum() + slac_sum_nse_cost

    cem_total_system_costs_sci = f"{cem_total_system_costs:.4e}"
    pf_total_system_costs_sci = f"{pf_total_system_costs:.4e}"
    dlac_total_system_costs_sci = f"{dlac_total_system_costs:.4e}"
    dlac_imperfect_total_system_costs_sci = f"{dlac_imperfect_total_system_costs:.4e}"
    slac_total_system_costs_sci = f"{slac_total_system_costs:.4e}"

    total_inv_op_costs = {
        'CEM': f"{cem_costs.sum().sum():.4e}",
        'PF': f"{pf_costs.sum().sum():.4e}",
        'DLAC': f"{dlac_costs.sum().sum():.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_costs.sum().sum():.4e}",
        'SLAC': f"{slac_costs.sum().sum():.4e}"
    }

    total_inv_op_costs_dic[case_name] = total_inv_op_costs

    # Create a dictionary with the total costs
    total_system_costs = {
        'CEM': cem_total_system_costs_sci,
        'PF': pf_total_system_costs_sci,
        'DLAC': dlac_total_system_costs_sci,
        'DLAC Imperfect': dlac_imperfect_total_system_costs_sci,
        'SLAC': slac_total_system_costs_sci
    }

    # # Sort the dictionary by values in descending order
    # sorted_total_system_costs = sorted(total_system_costs.items(), key=lambda item: item[1], reverse=True)

    # # Print the sorted total costs
    # for case, cost in sorted_total_system_costs:
    #     print("case_name: " f"{case}: {cost:.4e}")

    total_system_costs_dic[case_name] = total_system_costs

    ###================================================================================================
    ### Calculate Market Surplus/// Costs
    ###================================================================================================
    cem_objective_total = cem_total_system_costs + cem_sum_ur_cost
    pf_objective_total = pf_total_system_costs + pf_sum_ur_cost
    dlac_objective_total = dlac_total_system_costs + dlac_sum_ur_cost
    dlac_imperfect_objective_total = dlac_imperfect_total_system_costs + dlac_imperfect_sum_ur_cost
    slac_objective_total = slac_total_system_costs + slac_sum_ur_cost

    cem_objective_total_sci = f"{cem_objective_total:.4e}"
    pf_objective_total_sci = f"{pf_objective_total:.4e}"
    dlac_objective_total_sci = f"{dlac_objective_total:.4e}"
    dlac_imperfect_objective_total_sci = f"{dlac_imperfect_objective_total:.4e}"
    slac_objective_total_sci = f"{slac_objective_total:.4e}"

    # Create a dictionary with the total costs
    total_objective = {
        'CEM': cem_objective_total_sci,
        'PF': pf_objective_total_sci,
        'DLAC': dlac_objective_total_sci,
        'DLAC Imperfect': dlac_imperfect_objective_total_sci,
        'SLAC': slac_objective_total_sci
    }

    total_objective_dic[case_name] = total_objective

    ###================================================================================================
    ### Calculate Operating Costs
    ###================================================================================================
    operating_costs = ['Fixed_OM_cost_MW', 'Var_OM_cost_out', 'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost']

    cem_operating_costs = cem_costs[operating_costs].sum().sum()
    pf_operating_costs = pf_costs[operating_costs].sum().sum()
    dlac_operating_costs = dlac_costs[operating_costs].sum().sum()
    dlac_imperfect_operating_costs = dlac_imperfect_costs[operating_costs].sum().sum()
    slac_operating_costs = slac_costs[operating_costs].sum().sum()

    cem_operating_costs_sci = f"{cem_operating_costs:.4e}"
    pf_operating_costs_sci = f"{pf_operating_costs:.4e}"
    dlac_operating_costs_sci = f"{dlac_operating_costs:.4e}"
    dlac_imperfect_operating_costs_sci = f"{dlac_imperfect_operating_costs:.4e}"
    slac_operating_costs_sci = f"{slac_operating_costs:.4e}"

    # Create a dictionary with the operating costs
    total_operating_costs = {
        'CEM': cem_operating_costs_sci,
        'PF': pf_operating_costs_sci,
        'DLAC': dlac_operating_costs_sci,
        'DLAC Imperfect': dlac_imperfect_operating_costs_sci,
        'SLAC': slac_operating_costs_sci
    }

    total_operating_costs_dic[case_name] = total_operating_costs

    ###================================================================================================
    ### Calculate Fixed OM Costs
    ###================================================================================================

    cem_fixed_om_costs = cem_costs['Fixed_OM_cost_MW'].sum().sum()
    pf_fixed_om_costs = pf_costs['Fixed_OM_cost_MW'].sum().sum()
    dlac_fixed_om_costs = dlac_costs['Fixed_OM_cost_MW'].sum().sum()
    dlac_imperfect_fixed_om_costs = dlac_imperfect_costs['Fixed_OM_cost_MW'].sum().sum()
    slac_fixed_om_costs = slac_costs['Fixed_OM_cost_MW'].sum().sum()

    cem_fixed_om_costs_sci = f"{cem_fixed_om_costs:.4e}"
    pf_fixed_om_costs_sci = f"{pf_fixed_om_costs:.4e}"
    dlac_fixed_om_costs_sci = f"{dlac_fixed_om_costs:.4e}"
    dlac_imperfect_fixed_om_costs_sci = f"{dlac_imperfect_fixed_om_costs:.4e}"
    slac_fixed_om_costs_sci = f"{slac_fixed_om_costs:.4e}"

    # Create a dictionary with the fixed OM costs
    total_fixed_om_costs = {
        'CEM': cem_fixed_om_costs_sci,
        'PF': pf_fixed_om_costs_sci,
        'DLAC': dlac_fixed_om_costs_sci,
        'DLAC Imperfect': dlac_imperfect_fixed_om_costs_sci,
        'SLAC': slac_fixed_om_costs_sci
    }

    total_fixed_om_costs_dic[case_name] = total_fixed_om_costs

    ###================================================================================================
    ### Calculate Variable OM Costs
    ###================================================================================================

    cem_var_om_costs = cem_costs['Var_OM_cost_out'].sum().sum()
    pf_var_om_costs = pf_costs['Var_OM_cost_out'].sum().sum()
    dlac_var_om_costs = dlac_costs['Var_OM_cost_out'].sum().sum()
    dlac_imperfect_var_om_costs = dlac_imperfect_costs['Var_OM_cost_out'].sum().sum()
    slac_var_om_costs = slac_costs['Var_OM_cost_out'].sum().sum()

    cem_var_om_costs_sci = f"{cem_var_om_costs:.4e}"
    pf_var_om_costs_sci = f"{pf_var_om_costs:.4e}"
    dlac_var_om_costs_sci = f"{dlac_var_om_costs:.4e}"
    dlac_imperfect_var_om_costs_sci = f"{dlac_imperfect_var_om_costs:.4e}"
    slac_var_om_costs_sci = f"{slac_var_om_costs:.4e}"

    # Create a dictionary with the variable OM costs
    total_var_om_costs = {
        'CEM': cem_var_om_costs_sci,
        'PF': pf_var_om_costs_sci,
        'DLAC': dlac_var_om_costs_sci,
        'DLAC Imperfect': dlac_imperfect_var_om_costs_sci,
        'SLAC': slac_var_om_costs_sci
    }

    total_var_om_out_costs_dic[case_name] = total_var_om_costs

    ###================================================================================================
    ### Calculate Fuel Costs
    ###================================================================================================

    cem_fuel_costs = cem_costs['Fuel_cost'].sum().sum()
    pf_fuel_costs = pf_costs['Fuel_cost'].sum().sum()
    dlac_fuel_costs = dlac_costs['Fuel_cost'].sum().sum()
    dlac_imperfect_fuel_costs = dlac_imperfect_costs['Fuel_cost'].sum().sum()
    slac_fuel_costs = slac_costs['Fuel_cost'].sum().sum()

    cem_fuel_costs_sci = f"{cem_fuel_costs:.4e}"
    pf_fuel_costs_sci = f"{pf_fuel_costs:.4e}"
    dlac_fuel_costs_sci = f"{dlac_fuel_costs:.4e}"
    dlac_imperfect_fuel_costs_sci = f"{dlac_imperfect_fuel_costs:.4e}"
    slac_fuel_costs_sci = f"{slac_fuel_costs:.4e}"

    # Create a dictionary with the fuel costs
    total_fuel_costs = {
        'CEM': cem_fuel_costs_sci,
        'PF': pf_fuel_costs_sci,
        'DLAC': dlac_fuel_costs_sci,
        'DLAC Imperfect': dlac_imperfect_fuel_costs_sci,
        'SLAC': slac_fuel_costs_sci
    }

    total_fuel_costs_dic[case_name] = total_fuel_costs

    ###================================================================================================
    ### Calculate Variable OM Costs
    ###================================================================================================

    cem_var_om_in_costs = cem_costs['Var_OM_cost_in'].sum().sum()
    pf_var_om_in_costs = pf_costs['Var_OM_cost_in'].sum().sum()
    dlac_var_om_in_costs = dlac_costs['Var_OM_cost_in'].sum().sum()
    dlac_imperfect_var_om_in_costs = dlac_imperfect_costs['Var_OM_cost_in'].sum().sum()
    slac_var_om_in_costs = slac_costs['Var_OM_cost_in'].sum().sum()

    cem_var_om_in_costs_sci = f"{cem_var_om_in_costs:.4e}"
    pf_var_om_in_costs_sci = f"{pf_var_om_in_costs:.4e}"
    dlac_var_om_in_costs_sci = f"{dlac_var_om_in_costs:.4e}"
    dlac_imperfect_var_om_in_costs_sci = f"{dlac_imperfect_var_om_in_costs:.4e}"
    slac_var_om_in_costs_sci = f"{slac_var_om_in_costs:.4e}"

    # Create a dictionary with the variable OM costs
    total_var_om_in_costs = {
        'CEM': cem_var_om_in_costs_sci,
        'PF': pf_var_om_in_costs_sci,
        'DLAC': dlac_var_om_in_costs_sci,
        'DLAC Imperfect': dlac_imperfect_var_om_in_costs_sci,
        'SLAC': slac_var_om_in_costs_sci
    }

    total_var_om_in_costs_dic[case_name] = total_var_om_in_costs

    ###================================================================================================
    ### Calculate Start Costs
    ###================================================================================================

    cem_start_costs = cem_costs['StartCost'].sum().sum()
    pf_start_costs = pf_costs['StartCost'].sum().sum()
    dlac_start_costs = dlac_costs['StartCost'].sum().sum()
    dlac_imperfect_start_costs = dlac_imperfect_costs['StartCost'].sum().sum()
    slac_start_costs = slac_costs['StartCost'].sum().sum()

    cem_start_costs_sci = f"{cem_start_costs:.4e}"
    pf_start_costs_sci = f"{pf_start_costs:.4e}"
    dlac_start_costs_sci = f"{dlac_start_costs:.4e}"
    dlac_imperfect_start_costs_sci = f"{dlac_imperfect_start_costs:.4e}"
    slac_start_costs_sci = f"{slac_start_costs:.4e}"

    # Create a dictionary with the start costs
    total_start_costs = {
        'CEM': cem_start_costs_sci,
        'PF': pf_start_costs_sci,
        'DLAC': dlac_start_costs_sci,
        'DLAC Imperfect': dlac_imperfect_start_costs_sci,
        'SLAC': slac_start_costs_sci
    }

    total_start_costs_dic[case_name] = total_start_costs

    ###================================================================================================
    ### Calculate Charge Costs
    ###================================================================================================

    cem_charge_costs = cem_costs['Charge_cost'].sum().sum()
    pf_charge_costs = pf_costs['Charge_cost'].sum().sum()
    dlac_charge_costs = dlac_costs['Charge_cost'].sum().sum()
    dlac_imperfect_charge_costs = dlac_imperfect_costs['Charge_cost'].sum().sum()
    slac_charge_costs = slac_costs['Charge_cost'].sum().sum()

    cem_charge_costs_sci = f"{cem_charge_costs:.4e}"
    pf_charge_costs_sci = f"{pf_charge_costs:.4e}"
    dlac_charge_costs_sci = f"{dlac_charge_costs:.4e}"
    dlac_imperfect_charge_costs_sci = f"{dlac_imperfect_charge_costs:.4e}"
    slac_charge_costs_sci = f"{slac_charge_costs:.4e}"

    # Create a dictionary with the charge costs
    total_charge_costs = {
        'CEM': cem_charge_costs_sci,
        'PF': pf_charge_costs_sci,
        'DLAC': dlac_charge_costs_sci,
        'DLAC Imperfect': dlac_imperfect_charge_costs_sci,
        'SLAC': slac_charge_costs_sci
    }

    total_charge_costs_dic[case_name] = total_charge_costs

    ###================================================================================================
    ### Calculate Profit Margin Rate
    ###================================================================================================

    # dfGen.index = gen_print_names
    char_str = ['Inv_Cost_per_MWyr', 'Inv_Cost_per_MWhyr', 'Fixed_OM_Cost_per_MWyr']
    dfGen_annual_costs = dfGen[char_str]

    # if the Inv_Cost_per_MWhyr in each row is > 0, then multiply the Inv_Cost_per_MWhyr by the Max_Duration
    inv_cost_per_mwhyr2mwyr = dfGen.apply(
        lambda row: row['Inv_Cost_per_MWhyr'] * row['Existing_Cap_MWh'] / row['Existing_Cap_MW'] \
            if row['Inv_Cost_per_MWhyr'] > 0 else row['Inv_Cost_per_MWhyr'],
        axis=1
    )

    # Replace 'Inv_Cost_per_MWhyr' in dfGen_annual_costs with inv_cost_permwhyr2mwyr and title it 'Inv_Cost_per_Energy_by_hour'
    dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
    dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)

    dfGen_annual_costs_sum = dfGen_annual_costs.sum(axis=1) # $/MWyr

    # \alpha_{i,\Pi} = \frac{P_{i,\Pi}}{C_{i,\Pi}}
    cem_annual_profit2costs = cem_unit_profit_mw / dfGen_annual_costs_sum
    pf_annual_profit2costs = pf_unit_profit_mw / dfGen_annual_costs_sum
    dlac_annual_profit2costs = dlac_unit_profit_mw / dfGen_annual_costs_sum
    dlac_imperfect_annual_profit2costs = dlac_imperfect_unit_profit_mw / dfGen_annual_costs_sum
    slac_annual_profit2costs = slac_unit_profit_mw / dfGen_annual_costs_sum

    cem_ap2c_perc = cem_annual_profit2costs * 100
    pf_ap2c_perc = pf_annual_profit2costs * 100
    dlac_ap2c_perc = dlac_annual_profit2costs * 100
    dlac_imperfect_ap2c_perc = dlac_imperfect_annual_profit2costs * 100
    slac_ap2c_perc = slac_annual_profit2costs * 100

    cem_ap2c_perc_sci = cem_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    pf_ap2c_perc_sci = pf_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_ap2c_perc_sci = dlac_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_imperfect_perc_ap2c_sci = dlac_imperfect_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    slac_ap2c_perc_sci = slac_ap2c_perc.apply(lambda x: f"{x:.2f}%")


    percentage_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_ap2c_perc_sci,
        'PF': pf_ap2c_perc_sci,
        'DLAC': dlac_ap2c_perc_sci,
        'DLAC Imperfect': dlac_imperfect_perc_ap2c_sci,
        'SLAC': slac_ap2c_perc_sci
    })

    percentage_profits_dic[case_name] = percentage_profit_df
    ### DONE


Case Name: Thermal_Base



FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\ks885\\Documents\\aa_research\\Modeling\\spcm_genx_experiment\\SPCM\\research_systems\\Thermal_Base\\results_dlac-i\\NetRevenue.csv'

In [ ]:
def percent_change_df(df):
    norm = df.copy()
    norm['CEM'] = df['CEM'].astype(float) / df['PF'].astype(float)
    norm['PF'] = df['PF'].astype(float) / df['PF'].astype(float)
    norm['DLAC'] = df['DLAC'].astype(float) / df['PF'].astype(float)
    norm['DLAC Imperfect'] = df['DLAC Imperfect'].astype(float) / df['PF'].astype(float)
    norm['SLAC'] = df['SLAC'].astype(float) / df['PF'].astype(float)

    change_df = norm.copy()
    change_df['CEM'] = norm['CEM'] - 1
    change_df['PF'] = norm['PF'] - 1
    change_df['DLAC'] = norm['DLAC'] - 1
    change_df['DLAC Imperfect'] = norm['DLAC Imperfect'] - 1
    change_df['SLAC'] = norm['SLAC'] - 1

    percent_change_df = change_df.copy()
    percent_change_df['CEM'] = change_df['CEM'] * 100
    percent_change_df['PF'] = change_df['PF'] * 100
    percent_change_df['DLAC'] = change_df['DLAC'] * 100
    percent_change_df['DLAC Imperfect'] = change_df['DLAC Imperfect'] * 100
    percent_change_df['SLAC'] = change_df['SLAC'] * 100

    # remove decimal point
    percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
                        percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].round(2)
    percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
        percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

    return percent_change_df


'c:\\Users\\ks885\\Documents\\aa_research\\Modeling\\spcm_genx_experiment\\figures\\plots'

In [ ]:
# create dataframe from total_nse_dic with rows for each case_name in case_names
total_nse_df = pd.DataFrame.from_dict(total_nse_dic, orient='index').reset_index()
total_nse_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_nse_df.to_csv(tables_path + 'total_nse.csv', index=False)
total_nse_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,8.2823e+03,0.0000e+00,8.2823e+03,1.4377e+04,1.3434e+04
1,2_Hour_Batteries_Case,3.3547e+04,0.0000e+00,2.2896e+04,5.2365e+04,4.5367e+04
2,2_Hour_Batteries_Case_Fuelx2,2.8416e+04,0.0000e+00,1.9587e+04,6.6384e+04,5.3061e+04
3,4_Hour_Batteries_Case,2.3156e+04,0.0000e+00,1.4762e+04,5.0594e+04,4.3681e+04
4,4_Hour_Batteries_Case_Fuelx2,2.3021e+04,0.0000e+00,1.3930e+04,5.4732e+04,4.3961e+04
5,4_Hour_Batteries_Case_Fuelx3,1.0358e+04,0.0000e+00,1.2667e+03,4.6662e+04,4.6118e+04
6,4_Hour_Batteries_Case_Fuelx4,7.8305e+03,0.0000e+00,0.0000e+00,4.6476e+04,4.8763e+04
7,6_Hour_Batteries_Case,2.1136e+04,0.0000e+00,1.2356e+04,5.0969e+04,4.6695e+04
8,8_Hour_Batteries_Case,1.8288e+04,0.0000e+00,9.3269e+03,4.5519e+04,4.8563e+04
9,8_Hour_Batteries_Case_Fuelx2,5.3166e+03,0.0000e+00,0.0000e+00,3.7722e+04,3.6201e+04


In [ ]:
nse_change_df = percent_change_df(total_nse_df)
nse_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,inf%,nan%,inf%,inf%,inf%
1,2_Hour_Batteries_Case,inf%,nan%,inf%,inf%,inf%
2,2_Hour_Batteries_Case_Fuelx2,inf%,nan%,inf%,inf%,inf%
3,4_Hour_Batteries_Case,inf%,nan%,inf%,inf%,inf%
4,4_Hour_Batteries_Case_Fuelx2,inf%,nan%,inf%,inf%,inf%
5,4_Hour_Batteries_Case_Fuelx3,inf%,nan%,inf%,inf%,inf%
6,4_Hour_Batteries_Case_Fuelx4,inf%,nan%,nan%,inf%,inf%
7,6_Hour_Batteries_Case,inf%,nan%,inf%,inf%,inf%
8,8_Hour_Batteries_Case,inf%,nan%,inf%,inf%,inf%
9,8_Hour_Batteries_Case_Fuelx2,inf%,nan%,nan%,inf%,inf%


In [ ]:
total_unmet_rsv_df = pd.DataFrame.from_dict(total_unmet_rsv_dic, orient='index').reset_index()
total_unmet_rsv_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_unmet_rsv_df.to_csv(tables_path + 'total_unmet_rsv.csv', index=False)
total_unmet_rsv_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,3.5016e+04,0.0000e+00,3.6940e+04,3.7691e+04,3.6899e+04
1,2_Hour_Batteries_Case,6.7011e+04,0.0000e+00,4.6732e+03,2.4773e+04,2.7083e+04
2,2_Hour_Batteries_Case_Fuelx2,7.9461e+04,0.0000e+00,2.1122e+03,1.8335e+04,2.3521e+04
3,4_Hour_Batteries_Case,6.2786e+04,0.0000e+00,7.3229e+03,2.5837e+04,2.8946e+04
4,4_Hour_Batteries_Case_Fuelx2,8.4104e+04,0.0000e+00,1.9704e+03,1.3418e+04,1.5031e+04
5,4_Hour_Batteries_Case_Fuelx3,8.4104e+04,0.0000e+00,1.7863e+03,5.4765e+03,7.1595e+03
6,4_Hour_Batteries_Case_Fuelx4,8.4104e+04,0.0000e+00,2.5255e+02,3.1317e+03,4.6104e+03
7,6_Hour_Batteries_Case,6.4190e+04,0.0000e+00,9.5602e+03,2.3613e+04,3.2716e+04
8,8_Hour_Batteries_Case,6.3725e+04,0.0000e+00,2.0475e+04,2.4229e+04,3.2724e+04
9,8_Hour_Batteries_Case_Fuelx2,8.4104e+04,0.0000e+00,6.8445e+01,2.1815e+03,2.9207e+03


In [ ]:
# Create a dataframe from consumer_payments_dic with rows for each case_name in case_names
consumer_payments_df = pd.DataFrame.from_dict(consumer_payments_dic, orient='index').reset_index()
consumer_payments_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
consumer_payments_df.to_csv(tables_path + 'consumer_payments.csv', index=False)
consumer_payments_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,1.6515e+10,3.7214e+12,1.6302e+10,1.7394e+10,1.7034e+10
1,2_Hour_Batteries_Case,2.0175e+10,2.6610e+07,1.8215e+10,1.8392e+10,1.7809e+10
2,2_Hour_Batteries_Case_Fuelx2,2.4083e+10,2.3175e+07,2.1150e+10,2.0827e+10,2.2033e+10
3,4_Hour_Batteries_Case,2.0133e+10,4.1132e+07,1.7106e+10,1.7638e+10,1.7669e+10
4,4_Hour_Batteries_Case_Fuelx2,2.3960e+10,2.1504e+07,1.9694e+10,1.9597e+10,1.9955e+10
5,4_Hour_Batteries_Case_Fuelx3,2.6129e+10,2.0221e+07,2.2367e+10,1.9236e+10,2.1296e+10
6,4_Hour_Batteries_Case_Fuelx4,2.7822e+10,2.5805e+07,2.1572e+10,2.0522e+10,2.2194e+10
7,6_Hour_Batteries_Case,2.0130e+10,6.0954e+07,1.7244e+10,1.7936e+10,1.8574e+10
8,8_Hour_Batteries_Case,2.0143e+10,6.6676e+07,1.7821e+10,1.8223e+10,1.9907e+10
9,8_Hour_Batteries_Case_Fuelx2,2.3905e+10,3.1891e+07,1.6931e+10,1.8303e+10,1.8531e+10


In [ ]:
consumer_payments_change_df = percent_change_df(consumer_payments_df)
consumer_payments_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,-99.56%,0.0%,-99.56%,-99.53%,-99.54%
1,2_Hour_Batteries_Case,75717.36%,0.0%,68351.71%,69016.87%,66825.97%
2,2_Hour_Batteries_Case_Fuelx2,103818.02%,0.0%,91162.14%,89768.39%,94972.28%
3,4_Hour_Batteries_Case,48847.29%,0.0%,41488.06%,42781.45%,42856.82%
4,4_Hour_Batteries_Case_Fuelx2,111321.13%,0.0%,91482.96%,91031.88%,92696.69%
5,4_Hour_Batteries_Case_Fuelx3,129117.15%,0.0%,110512.73%,95028.83%,105216.26%
6,4_Hour_Batteries_Case_Fuelx4,107716.31%,0.0%,83496.2%,79427.22%,85906.59%
7,6_Hour_Batteries_Case,32924.9%,0.0%,28190.19%,29325.47%,30372.16%
8,8_Hour_Batteries_Case,30110.27%,0.0%,26627.76%,27230.67%,29756.32%
9,8_Hour_Batteries_Case_Fuelx2,74858.45%,0.0%,52990.21%,57292.37%,58007.3%


In [ ]:
total_inv_costs_df = pd.DataFrame.from_dict(total_investment_costs_dic, orient='index').reset_index()
total_inv_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_inv_costs_df.to_csv(tables_path + 'total_inv_costs.csv', index=False)
total_inv_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,6.8115e+09,6.8115e+09,6.8115e+09,6.8115e+09,6.8115e+09
1,2_Hour_Batteries_Case,9.5159e+09,9.5159e+09,9.5159e+09,9.5159e+09,9.5159e+09
2,2_Hour_Batteries_Case_Fuelx2,1.2467e+10,1.2467e+10,1.2467e+10,1.2467e+10,1.2467e+10
3,4_Hour_Batteries_Case,9.5397e+09,9.5397e+09,9.5397e+09,9.5397e+09,9.5397e+09
4,4_Hour_Batteries_Case_Fuelx2,1.2696e+10,1.2696e+10,1.2696e+10,1.2696e+10,1.2696e+10
5,4_Hour_Batteries_Case_Fuelx3,1.4195e+10,1.4195e+10,1.4195e+10,1.4195e+10,1.4195e+10
6,4_Hour_Batteries_Case_Fuelx4,1.5089e+10,1.5089e+10,1.5089e+10,1.5089e+10,1.5089e+10
7,6_Hour_Batteries_Case,9.6202e+09,9.6202e+09,9.6202e+09,9.6202e+09,9.6202e+09
8,8_Hour_Batteries_Case,9.6579e+09,9.6579e+09,9.6579e+09,9.6579e+09,9.6579e+09
9,8_Hour_Batteries_Case_Fuelx2,1.2942e+10,1.2942e+10,1.2942e+10,1.2942e+10,1.2942e+10


In [ ]:
total_inv_cost_change = percent_change_df(total_inv_costs_df)
total_inv_cost_change

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0%,0.0%,0.0%,0.0%,0.0%
1,2_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
2,2_Hour_Batteries_Case_Fuelx2,0.0%,0.0%,0.0%,0.0%,0.0%
3,4_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
4,4_Hour_Batteries_Case_Fuelx2,0.0%,0.0%,0.0%,0.0%,0.0%
5,4_Hour_Batteries_Case_Fuelx3,0.0%,0.0%,0.0%,0.0%,0.0%
6,4_Hour_Batteries_Case_Fuelx4,0.0%,0.0%,0.0%,0.0%,0.0%
7,6_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
8,8_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
9,8_Hour_Batteries_Case_Fuelx2,0.0%,0.0%,0.0%,0.0%,0.0%


In [ ]:
total_inv_op_costs_df = pd.DataFrame.from_dict(total_inv_op_costs_dic, orient='index').reset_index()
total_inv_op_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_inv_op_costs_df.to_csv(tables_path + 'total_inv_op_costs.csv', index=False)
total_inv_op_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,1.6474e+10,1.6462e+10,1.6468e+10,1.6310e+10,1.6300e+10
1,2_Hour_Batteries_Case,2.0184e+10,1.9798e+10,1.9811e+10,1.9582e+10,1.9581e+10
2,2_Hour_Batteries_Case_Fuelx2,2.4245e+10,2.3544e+10,2.3558e+10,2.3229e+10,2.3243e+10
3,4_Hour_Batteries_Case,2.0209e+10,1.9791e+10,1.9807e+10,1.9598e+10,1.9611e+10
4,4_Hour_Batteries_Case_Fuelx2,2.4218e+10,2.3485e+10,2.3520e+10,2.3183e+10,2.3210e+10
5,4_Hour_Batteries_Case_Fuelx3,2.6551e+10,2.5537e+10,2.5602e+10,2.5172e+10,2.5122e+10
6,4_Hour_Batteries_Case_Fuelx4,2.8364e+10,2.6954e+10,2.7012e+10,2.6495e+10,2.6396e+10
7,6_Hour_Batteries_Case,2.0223e+10,1.9810e+10,1.9826e+10,1.9626e+10,1.9651e+10
8,8_Hour_Batteries_Case,2.0255e+10,1.9836e+10,1.9858e+10,1.9671e+10,1.9696e+10
9,8_Hour_Batteries_Case_Fuelx2,2.4248e+10,2.3545e+10,2.3641e+10,2.3272e+10,2.3305e+10


In [ ]:
# Create a dataframe from total_system_costs_dic with rows for each case_name in case_names
total_operating_costs_df = pd.DataFrame.from_dict(total_operating_costs_dic, orient='index').reset_index()
total_operating_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_operating_costs_df.to_csv(tables_path + 'total_operational_costs.csv', index=False)
total_operating_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,9.6625e+09,9.6505e+09,9.6563e+09,9.4989e+09,9.4882e+09
1,2_Hour_Batteries_Case,1.0668e+10,1.0282e+10,1.0295e+10,1.0066e+10,1.0065e+10
2,2_Hour_Batteries_Case_Fuelx2,1.1778e+10,1.1077e+10,1.1091e+10,1.0762e+10,1.0776e+10
3,4_Hour_Batteries_Case,1.0669e+10,1.0252e+10,1.0267e+10,1.0058e+10,1.0071e+10
4,4_Hour_Batteries_Case_Fuelx2,1.1522e+10,1.0788e+10,1.0824e+10,1.0486e+10,1.0513e+10
5,4_Hour_Batteries_Case_Fuelx3,1.2356e+10,1.1342e+10,1.1407e+10,1.0978e+10,1.0927e+10
6,4_Hour_Batteries_Case_Fuelx4,1.3274e+10,1.1864e+10,1.1923e+10,1.1405e+10,1.1307e+10
7,6_Hour_Batteries_Case,1.0602e+10,1.0190e+10,1.0206e+10,1.0005e+10,1.0031e+10
8,8_Hour_Batteries_Case,1.0597e+10,1.0178e+10,1.0200e+10,1.0013e+10,1.0038e+10
9,8_Hour_Batteries_Case_Fuelx2,1.1306e+10,1.0603e+10,1.0699e+10,1.0331e+10,1.0363e+10


In [ ]:
operating_cost_change_df = percent_change_df(total_operating_costs_df)
operating_cost_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.12%,0.0%,0.06%,-1.57%,-1.68%
1,2_Hour_Batteries_Case,3.75%,0.0%,0.13%,-2.1%,-2.11%
2,2_Hour_Batteries_Case_Fuelx2,6.33%,0.0%,0.13%,-2.84%,-2.72%
3,4_Hour_Batteries_Case,4.07%,0.0%,0.15%,-1.89%,-1.77%
4,4_Hour_Batteries_Case_Fuelx2,6.8%,0.0%,0.33%,-2.8%,-2.55%
5,4_Hour_Batteries_Case_Fuelx3,8.94%,0.0%,0.57%,-3.21%,-3.66%
6,4_Hour_Batteries_Case_Fuelx4,11.88%,0.0%,0.5%,-3.87%,-4.69%
7,6_Hour_Batteries_Case,4.04%,0.0%,0.16%,-1.82%,-1.56%
8,8_Hour_Batteries_Case,4.12%,0.0%,0.22%,-1.62%,-1.38%
9,8_Hour_Batteries_Case_Fuelx2,6.63%,0.0%,0.91%,-2.57%,-2.26%


In [ ]:
total_fixed_om_costs_df = pd.DataFrame.from_dict(total_fixed_om_costs_dic, orient='index').reset_index()
total_fixed_om_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_fixed_om_costs_df.to_csv(tables_path + 'total_fixed_om_costs.csv', index=False)
total_fixed_om_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,3.0089e+09,3.0089e+09,3.0089e+09,3.0089e+09,3.0089e+09
1,2_Hour_Batteries_Case,4.1676e+09,4.1676e+09,4.1676e+09,4.1676e+09,4.1676e+09
2,2_Hour_Batteries_Case_Fuelx2,5.2662e+09,5.2662e+09,5.2662e+09,5.2662e+09,5.2662e+09
3,4_Hour_Batteries_Case,4.1598e+09,4.1598e+09,4.1598e+09,4.1598e+09,4.1598e+09
4,4_Hour_Batteries_Case_Fuelx2,5.2545e+09,5.2545e+09,5.2545e+09,5.2545e+09,5.2545e+09
5,4_Hour_Batteries_Case_Fuelx3,5.6716e+09,5.6716e+09,5.6716e+09,5.6716e+09,5.6716e+09
6,4_Hour_Batteries_Case_Fuelx4,5.9633e+09,5.9633e+09,5.9633e+09,5.9633e+09,5.9633e+09
7,6_Hour_Batteries_Case,4.1466e+09,4.1466e+09,4.1466e+09,4.1466e+09,4.1466e+09
8,8_Hour_Batteries_Case,4.1289e+09,4.1289e+09,4.1289e+09,4.1289e+09,4.1289e+09
9,8_Hour_Batteries_Case_Fuelx2,5.1907e+09,5.1907e+09,5.1907e+09,5.1907e+09,5.1907e+09


In [ ]:
total_fixed_om_costs_change_df = percent_change_df(total_fixed_om_costs_df)
total_fixed_om_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0%,0.0%,0.0%,0.0%,0.0%
1,2_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
2,2_Hour_Batteries_Case_Fuelx2,0.0%,0.0%,0.0%,0.0%,0.0%
3,4_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
4,4_Hour_Batteries_Case_Fuelx2,0.0%,0.0%,0.0%,0.0%,0.0%
5,4_Hour_Batteries_Case_Fuelx3,0.0%,0.0%,0.0%,0.0%,0.0%
6,4_Hour_Batteries_Case_Fuelx4,0.0%,0.0%,0.0%,0.0%,0.0%
7,6_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
8,8_Hour_Batteries_Case,0.0%,0.0%,0.0%,0.0%,0.0%
9,8_Hour_Batteries_Case_Fuelx2,0.0%,0.0%,0.0%,0.0%,0.0%


In [ ]:
total_var_om_out_costs_df = pd.DataFrame.from_dict(total_var_om_out_costs_dic, orient='index').reset_index()
total_var_om_out_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_var_om_out_costs_df.to_csv(tables_path + 'total_var_om_out_costs.csv', index=False)
total_var_om_out_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,3.6207e+08,3.6174e+08,3.6210e+08,3.5330e+08,3.5126e+08
1,2_Hour_Batteries_Case,5.4735e+08,4.9752e+08,4.9767e+08,4.7754e+08,4.7614e+08
2,2_Hour_Batteries_Case_Fuelx2,5.3983e+08,4.4975e+08,4.4937e+08,4.1996e+08,4.1559e+08
3,4_Hour_Batteries_Case,5.4633e+08,4.9618e+08,4.9688e+08,4.7713e+08,4.7677e+08
4,4_Hour_Batteries_Case_Fuelx2,5.2503e+08,4.3241e+08,4.3359e+08,4.0604e+08,4.0144e+08
5,4_Hour_Batteries_Case_Fuelx3,4.8560e+08,3.5980e+08,3.6134e+08,3.3633e+08,3.2681e+08
6,4_Hour_Batteries_Case_Fuelx4,4.6436e+08,3.1340e+08,3.1566e+08,2.9085e+08,2.7857e+08
7,6_Hour_Batteries_Case,5.4151e+08,4.9060e+08,4.9115e+08,4.7205e+08,4.7213e+08
8,8_Hour_Batteries_Case,5.4145e+08,4.9071e+08,4.9128e+08,4.7301e+08,4.7310e+08
9,8_Hour_Batteries_Case_Fuelx2,5.1491e+08,4.2014e+08,4.2212e+08,3.9531e+08,3.9406e+08


In [ ]:
total_var_om_out_costs_change_df = percent_change_df(total_var_om_out_costs_df)
total_var_om_out_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.09%,0.0%,0.1%,-2.33%,-2.9%
1,2_Hour_Batteries_Case,10.02%,0.0%,0.03%,-4.02%,-4.3%
2,2_Hour_Batteries_Case_Fuelx2,20.03%,0.0%,-0.08%,-6.62%,-7.6%
3,4_Hour_Batteries_Case,10.11%,0.0%,0.14%,-3.84%,-3.91%
4,4_Hour_Batteries_Case_Fuelx2,21.42%,0.0%,0.27%,-6.1%,-7.16%
5,4_Hour_Batteries_Case_Fuelx3,34.96%,0.0%,0.43%,-6.52%,-9.17%
6,4_Hour_Batteries_Case_Fuelx4,48.17%,0.0%,0.72%,-7.2%,-11.11%
7,6_Hour_Batteries_Case,10.38%,0.0%,0.11%,-3.78%,-3.76%
8,8_Hour_Batteries_Case,10.34%,0.0%,0.12%,-3.61%,-3.59%
9,8_Hour_Batteries_Case_Fuelx2,22.56%,0.0%,0.47%,-5.91%,-6.21%


In [ ]:
total_fuel_costs_df = pd.DataFrame.from_dict(total_fuel_costs_dic, orient='index').reset_index()
total_fuel_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_fuel_costs_df.to_csv(tables_path + 'total_fuel_costs.csv', index=False)
total_fuel_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,6.0526e+09,6.0387e+09,6.0441e+09,5.8808e+09,5.8692e+09
1,2_Hour_Batteries_Case,5.6421e+09,5.3282e+09,5.3296e+09,5.1015e+09,5.0836e+09
2,2_Hour_Batteries_Case_Fuelx2,5.5405e+09,4.9831e+09,4.9933e+09,4.6583e+09,4.6137e+09
3,4_Hour_Batteries_Case,5.6345e+09,5.3149e+09,5.3192e+09,5.0969e+09,5.0898e+09
4,4_Hour_Batteries_Case_Fuelx2,5.2575e+09,4.6691e+09,4.6841e+09,4.3577e+09,4.2984e+09
5,4_Hour_Batteries_Case_Fuelx3,5.6351e+09,4.7076e+09,4.7300e+09,4.3086e+09,4.1886e+09
6,4_Hour_Batteries_Case_Fuelx4,6.1824e+09,4.8591e+09,4.8825e+09,4.4369e+09,4.2537e+09
7,6_Hour_Batteries_Case,5.5821e+09,5.2610e+09,5.2637e+09,5.0446e+09,5.0448e+09
8,8_Hour_Batteries_Case,5.5856e+09,5.2652e+09,5.2680e+09,5.0565e+09,5.0582e+09
9,8_Hour_Batteries_Case_Fuelx2,5.1235e+09,4.5333e+09,4.5671e+09,4.2119e+09,4.1583e+09


In [ ]:
total_fuel_costs_change_df = percent_change_df(total_fuel_costs_df)
total_fuel_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.23%,0.0%,0.09%,-2.61%,-2.81%
1,2_Hour_Batteries_Case,5.89%,0.0%,0.03%,-4.25%,-4.59%
2,2_Hour_Batteries_Case_Fuelx2,11.19%,0.0%,0.2%,-6.52%,-7.41%
3,4_Hour_Batteries_Case,6.01%,0.0%,0.08%,-4.1%,-4.24%
4,4_Hour_Batteries_Case_Fuelx2,12.6%,0.0%,0.32%,-6.67%,-7.94%
5,4_Hour_Batteries_Case_Fuelx3,19.7%,0.0%,0.48%,-8.48%,-11.02%
6,4_Hour_Batteries_Case_Fuelx4,27.23%,0.0%,0.48%,-8.69%,-12.46%
7,6_Hour_Batteries_Case,6.1%,0.0%,0.05%,-4.11%,-4.11%
8,8_Hour_Batteries_Case,6.09%,0.0%,0.05%,-3.96%,-3.93%
9,8_Hour_Batteries_Case_Fuelx2,13.02%,0.0%,0.75%,-7.09%,-8.27%


In [ ]:
total_var_om_in_costs_df = pd.DataFrame.from_dict(total_var_om_in_costs_dic, orient='index').reset_index()
total_var_om_in_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_var_om_in_costs_df.to_csv(tables_path + 'total_var_om_in_costs.csv', index=False)
total_var_om_in_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
1,2_Hour_Batteries_Case,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
2,2_Hour_Batteries_Case_Fuelx2,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
3,4_Hour_Batteries_Case,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
4,4_Hour_Batteries_Case_Fuelx2,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
5,4_Hour_Batteries_Case_Fuelx3,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
6,4_Hour_Batteries_Case_Fuelx4,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
7,6_Hour_Batteries_Case,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
8,8_Hour_Batteries_Case,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
9,8_Hour_Batteries_Case_Fuelx2,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00


In [ ]:
total_var_om_in_costs_change_df = percent_change_df(total_var_om_in_costs_df)
total_var_om_in_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,nan%,nan%,nan%,nan%,nan%
1,2_Hour_Batteries_Case,nan%,nan%,nan%,nan%,nan%
2,2_Hour_Batteries_Case_Fuelx2,nan%,nan%,nan%,nan%,nan%
3,4_Hour_Batteries_Case,nan%,nan%,nan%,nan%,nan%
4,4_Hour_Batteries_Case_Fuelx2,nan%,nan%,nan%,nan%,nan%
5,4_Hour_Batteries_Case_Fuelx3,nan%,nan%,nan%,nan%,nan%
6,4_Hour_Batteries_Case_Fuelx4,nan%,nan%,nan%,nan%,nan%
7,6_Hour_Batteries_Case,nan%,nan%,nan%,nan%,nan%
8,8_Hour_Batteries_Case,nan%,nan%,nan%,nan%,nan%
9,8_Hour_Batteries_Case_Fuelx2,nan%,nan%,nan%,nan%,nan%


In [ ]:
total_inv_op_costs_change_df = percent_change_df(total_inv_op_costs_df)
total_inv_op_costs_change_df.to_csv(tables_path + 'total_inv_op_costs_change.csv', index=False)
total_inv_op_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.07%,0.0%,0.04%,-0.92%,-0.98%
1,2_Hour_Batteries_Case,1.95%,0.0%,0.07%,-1.09%,-1.1%
2,2_Hour_Batteries_Case_Fuelx2,2.98%,0.0%,0.06%,-1.34%,-1.28%
3,4_Hour_Batteries_Case,2.11%,0.0%,0.08%,-0.98%,-0.91%
4,4_Hour_Batteries_Case_Fuelx2,3.12%,0.0%,0.15%,-1.29%,-1.17%
5,4_Hour_Batteries_Case_Fuelx3,3.97%,0.0%,0.25%,-1.43%,-1.63%
6,4_Hour_Batteries_Case_Fuelx4,5.23%,0.0%,0.22%,-1.7%,-2.07%
7,6_Hour_Batteries_Case,2.08%,0.0%,0.08%,-0.93%,-0.8%
8,8_Hour_Batteries_Case,2.11%,0.0%,0.11%,-0.83%,-0.71%
9,8_Hour_Batteries_Case_Fuelx2,2.99%,0.0%,0.41%,-1.16%,-1.02%


In [ ]:
total_unmet_rsv_cost_df = pd.DataFrame.from_dict(total_unmet_rsv_cost_dic, orient='index').reset_index()
total_unmet_rsv_cost_df.columns = ['Case Name', 'PF', 'CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_unmet_rsv_cost_df.to_csv(tables_path + 'total_unmet_rsv_costs.csv', index=False)
total_unmet_rsv_cost_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,3.5016e+07,0.0000e+00,3.6940e+07,3.7691e+07,3.69e+07
1,2_Hour_Batteries_Case,6.7011e+07,0.0000e+00,4.6732e+06,2.4773e+07,2.71e+07
2,2_Hour_Batteries_Case_Fuelx2,7.9461e+07,0.0000e+00,2.1122e+06,1.8335e+07,2.35e+07
3,4_Hour_Batteries_Case,6.2786e+07,0.0000e+00,7.3229e+06,2.5837e+07,2.89e+07
4,4_Hour_Batteries_Case_Fuelx2,8.4104e+07,0.0000e+00,1.9704e+06,1.3418e+07,1.50e+07
5,4_Hour_Batteries_Case_Fuelx3,8.4104e+07,0.0000e+00,1.7863e+06,5.4765e+06,7.16e+06
6,4_Hour_Batteries_Case_Fuelx4,8.4104e+07,0.0000e+00,2.5255e+05,3.1317e+06,4.61e+06
7,6_Hour_Batteries_Case,6.4190e+07,0.0000e+00,9.5602e+06,2.3613e+07,3.27e+07
8,8_Hour_Batteries_Case,6.3725e+07,0.0000e+00,2.0475e+07,2.4229e+07,3.27e+07
9,8_Hour_Batteries_Case_Fuelx2,8.4104e+07,0.0000e+00,6.8445e+04,2.1815e+06,2.92e+06


In [ ]:
total_change_unmet_rsv = percent_change_df(total_unmet_rsv_cost_df)
total_change_unmet_rsv

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0%,-100.0%,5.49%,7.64%,5.38%
1,2_Hour_Batteries_Case,0.0%,-100.0%,-93.03%,-63.03%,-59.56%
2,2_Hour_Batteries_Case_Fuelx2,0.0%,-100.0%,-97.34%,-76.93%,-70.43%
3,4_Hour_Batteries_Case,0.0%,-100.0%,-88.34%,-58.85%,-53.97%
4,4_Hour_Batteries_Case_Fuelx2,0.0%,-100.0%,-97.66%,-84.05%,-82.16%
5,4_Hour_Batteries_Case_Fuelx3,0.0%,-100.0%,-97.88%,-93.49%,-91.49%
6,4_Hour_Batteries_Case_Fuelx4,0.0%,-100.0%,-99.7%,-96.28%,-94.52%
7,6_Hour_Batteries_Case,0.0%,-100.0%,-85.11%,-63.21%,-49.06%
8,8_Hour_Batteries_Case,0.0%,-100.0%,-67.87%,-61.98%,-48.69%
9,8_Hour_Batteries_Case_Fuelx2,0.0%,-100.0%,-99.92%,-97.41%,-96.53%


In [ ]:
total_nse_cost_df = pd.DataFrame.from_dict(total_nse_cost_dic, orient='index').reset_index()
total_nse_cost_df.columns = ['Case Name', 'PF', 'CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_nse_cost_df.to_csv(tables_path + 'total_nse_costs.csv', index=False)
total_nse_cost_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,4.1411e+07,0.0000e+00,4.1411e+07,7.1884e+07,6.7169e+07
1,2_Hour_Batteries_Case,1.6773e+08,0.0000e+00,1.1448e+08,2.6183e+08,2.2683e+08
2,2_Hour_Batteries_Case_Fuelx2,1.4208e+08,0.0000e+00,9.7934e+07,3.3192e+08,2.6530e+08
3,4_Hour_Batteries_Case,1.1578e+08,0.0000e+00,7.3809e+07,2.5297e+08,2.1841e+08
4,4_Hour_Batteries_Case_Fuelx2,1.1510e+08,0.0000e+00,6.9649e+07,2.7366e+08,2.1981e+08
5,4_Hour_Batteries_Case_Fuelx3,5.1789e+07,0.0000e+00,6.3336e+06,2.3331e+08,2.3059e+08
6,4_Hour_Batteries_Case_Fuelx4,3.9153e+07,0.0000e+00,0.0000e+00,2.3238e+08,2.4381e+08
7,6_Hour_Batteries_Case,1.0568e+08,0.0000e+00,6.1780e+07,2.5485e+08,2.3347e+08
8,8_Hour_Batteries_Case,9.1440e+07,0.0000e+00,4.6635e+07,2.2759e+08,2.4281e+08
9,8_Hour_Batteries_Case_Fuelx2,2.6583e+07,0.0000e+00,0.0000e+00,1.8861e+08,1.8100e+08


In [ ]:
nse_cost_change_df = percent_change_df(total_nse_cost_df)
nse_cost_change_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0%,-100.0%,0.0%,73.59%,62.2%
1,2_Hour_Batteries_Case,0.0%,-100.0%,-31.75%,56.1%,35.24%
2,2_Hour_Batteries_Case_Fuelx2,0.0%,-100.0%,-31.07%,133.61%,86.73%
3,4_Hour_Batteries_Case,0.0%,-100.0%,-36.25%,118.49%,88.64%
4,4_Hour_Batteries_Case_Fuelx2,0.0%,-100.0%,-39.49%,137.76%,90.97%
5,4_Hour_Batteries_Case_Fuelx3,0.0%,-100.0%,-87.77%,350.5%,345.25%
6,4_Hour_Batteries_Case_Fuelx4,0.0%,-100.0%,-100.0%,493.52%,522.71%
7,6_Hour_Batteries_Case,0.0%,-100.0%,-41.54%,141.15%,120.92%
8,8_Hour_Batteries_Case,0.0%,-100.0%,-49.0%,148.9%,165.54%
9,8_Hour_Batteries_Case_Fuelx2,0.0%,-100.0%,-100.0%,609.51%,580.89%


In [ ]:
# Create a dataframe from total_system_costs_dic with rows for each case_name in case_names
total_system_costs_df = pd.DataFrame.from_dict(total_system_costs_dic, orient='index').reset_index()
total_system_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_system_costs_df.to_csv(tables_path + 'total_system_costs.csv', index=False)
total_system_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,1.6515e+10,1.6462e+10,1.6509e+10,1.6382e+10,1.6367e+10
1,2_Hour_Batteries_Case,2.0352e+10,1.9798e+10,1.9925e+10,1.9844e+10,1.9808e+10
2,2_Hour_Batteries_Case_Fuelx2,2.4387e+10,2.3544e+10,2.3656e+10,2.3561e+10,2.3508e+10
3,4_Hour_Batteries_Case,2.0325e+10,1.9791e+10,1.9880e+10,1.9851e+10,1.9829e+10
4,4_Hour_Batteries_Case_Fuelx2,2.4333e+10,2.3485e+10,2.3590e+10,2.3456e+10,2.3429e+10
5,4_Hour_Batteries_Case_Fuelx3,2.6603e+10,2.5537e+10,2.5608e+10,2.5406e+10,2.5352e+10
6,4_Hour_Batteries_Case_Fuelx4,2.8403e+10,2.6954e+10,2.7012e+10,2.6727e+10,2.6640e+10
7,6_Hour_Batteries_Case,2.0328e+10,1.9810e+10,1.9888e+10,1.9880e+10,1.9884e+10
8,8_Hour_Batteries_Case,2.0346e+10,1.9836e+10,1.9905e+10,1.9899e+10,1.9939e+10
9,8_Hour_Batteries_Case_Fuelx2,2.4275e+10,2.3545e+10,2.3641e+10,2.3461e+10,2.3486e+10


In [ ]:
total_system_costs_change_df = percent_change_df(total_system_costs_df)
total_system_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.32%,0.0%,0.29%,-0.49%,-0.58%
1,2_Hour_Batteries_Case,2.8%,0.0%,0.64%,0.23%,0.05%
2,2_Hour_Batteries_Case_Fuelx2,3.58%,0.0%,0.48%,0.07%,-0.15%
3,4_Hour_Batteries_Case,2.7%,0.0%,0.45%,0.3%,0.19%
4,4_Hour_Batteries_Case_Fuelx2,3.61%,0.0%,0.45%,-0.12%,-0.24%
5,4_Hour_Batteries_Case_Fuelx3,4.17%,0.0%,0.28%,-0.51%,-0.72%
6,4_Hour_Batteries_Case_Fuelx4,5.38%,0.0%,0.22%,-0.84%,-1.16%
7,6_Hour_Batteries_Case,2.61%,0.0%,0.39%,0.35%,0.37%
8,8_Hour_Batteries_Case,2.57%,0.0%,0.35%,0.32%,0.52%
9,8_Hour_Batteries_Case_Fuelx2,3.1%,0.0%,0.41%,-0.36%,-0.25%


In [ ]:
total_objective_df = pd.DataFrame.from_dict(total_objective_dic, orient='index').reset_index()
total_objective_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_objective_df.to_csv(tables_path + 'total_objective.csv', index=False)
total_objective_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,1.6550e+10,1.6462e+10,1.6546e+10,1.6420e+10,1.6404e+10
1,2_Hour_Batteries_Case,2.0419e+10,1.9798e+10,1.9930e+10,1.9869e+10,1.9835e+10
2,2_Hour_Batteries_Case_Fuelx2,2.4467e+10,2.3544e+10,2.3658e+10,2.3579e+10,2.3532e+10
3,4_Hour_Batteries_Case,2.0388e+10,1.9791e+10,1.9888e+10,1.9877e+10,1.9858e+10
4,4_Hour_Batteries_Case_Fuelx2,2.4417e+10,2.3485e+10,2.3592e+10,2.3470e+10,2.3444e+10
5,4_Hour_Batteries_Case_Fuelx3,2.6687e+10,2.5537e+10,2.5610e+10,2.5411e+10,2.5359e+10
6,4_Hour_Batteries_Case_Fuelx4,2.8487e+10,2.6954e+10,2.7012e+10,2.6730e+10,2.6644e+10
7,6_Hour_Batteries_Case,2.0392e+10,1.9810e+10,1.9897e+10,1.9904e+10,1.9917e+10
8,8_Hour_Batteries_Case,2.0410e+10,1.9836e+10,1.9925e+10,1.9923e+10,1.9972e+10
9,8_Hour_Batteries_Case_Fuelx2,2.4359e+10,2.3545e+10,2.3641e+10,2.3463e+10,2.3488e+10


In [ ]:
total_objective_change_df = percent_change_df(total_objective_df)
total_objective_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.53%,0.0%,0.51%,-0.26%,-0.35%
1,2_Hour_Batteries_Case,3.14%,0.0%,0.67%,0.36%,0.19%
2,2_Hour_Batteries_Case_Fuelx2,3.92%,0.0%,0.48%,0.15%,-0.05%
3,4_Hour_Batteries_Case,3.02%,0.0%,0.49%,0.43%,0.34%
4,4_Hour_Batteries_Case_Fuelx2,3.97%,0.0%,0.46%,-0.06%,-0.17%
5,4_Hour_Batteries_Case_Fuelx3,4.5%,0.0%,0.29%,-0.49%,-0.7%
6,4_Hour_Batteries_Case_Fuelx4,5.69%,0.0%,0.22%,-0.83%,-1.15%
7,6_Hour_Batteries_Case,2.94%,0.0%,0.44%,0.47%,0.54%
8,8_Hour_Batteries_Case,2.89%,0.0%,0.45%,0.44%,0.69%
9,8_Hour_Batteries_Case_Fuelx2,3.46%,0.0%,0.41%,-0.35%,-0.24%


### These nse costs are calculated exogenously

In [ ]:

# multiply the nse_totals by 5000 to get the total nse costs and compare against the total_nse_cost_df
nse_cost_from_totals = total_nse_df.copy()
for col in total_nse_cost_df.columns:
    if col != 'Case Name':
        nse_cost_from_totals[col] = nse_cost_from_totals[col].astype(float) * 5000

# Convert the values to scientific notation
for col in nse_cost_from_totals.columns:
    if col != 'Case Name':
        nse_cost_from_totals[col] = nse_cost_from_totals[col].apply(lambda x: f"{x:.4e}")

nse_cost_from_totals

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,4.1412e+07,0.0000e+00,4.1412e+07,7.1885e+07,6.7170e+07
1,2_Hour_Batteries_Case,1.6774e+08,0.0000e+00,1.1448e+08,2.6182e+08,2.2684e+08
2,2_Hour_Batteries_Case_Fuelx2,1.4208e+08,0.0000e+00,9.7935e+07,3.3192e+08,2.6530e+08
3,4_Hour_Batteries_Case,1.1578e+08,0.0000e+00,7.3810e+07,2.5297e+08,2.1840e+08
4,4_Hour_Batteries_Case_Fuelx2,1.1510e+08,0.0000e+00,6.9650e+07,2.7366e+08,2.1980e+08
5,4_Hour_Batteries_Case_Fuelx3,5.1790e+07,0.0000e+00,6.3335e+06,2.3331e+08,2.3059e+08
6,4_Hour_Batteries_Case_Fuelx4,3.9152e+07,0.0000e+00,0.0000e+00,2.3238e+08,2.4382e+08
7,6_Hour_Batteries_Case,1.0568e+08,0.0000e+00,6.1780e+07,2.5484e+08,2.3348e+08
8,8_Hour_Batteries_Case,9.1440e+07,0.0000e+00,4.6634e+07,2.2760e+08,2.4282e+08
9,8_Hour_Batteries_Case_Fuelx2,2.6583e+07,0.0000e+00,0.0000e+00,1.8861e+08,1.8100e+08


In [ ]:
total_operating_costs_norm = total_operating_costs_df.copy()

total_operating_costs_norm['CEM'] = total_operating_costs_norm['CEM'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['PF'] = total_operating_costs_norm['PF'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['DLAC'] = total_operating_costs_norm['DLAC'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['DLAC Imperfect'] = total_operating_costs_norm['DLAC Imperfect'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['SLAC'] = total_operating_costs_norm['SLAC'].astype(float) / total_operating_costs_df['CEM'].astype(float)

total_operating_costs_norm.to_csv(tables_path + 'total_operating_costs_norm.csv', index=False)
total_operating_costs_norm

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,1.0,0.998758,0.999358,0.983069,0.981961
1,2_Hour_Batteries_Case,1.0,0.963817,0.965036,0.943570,0.943476
2,2_Hour_Batteries_Case_Fuelx2,1.0,0.940482,0.941671,0.913737,0.914926
3,4_Hour_Batteries_Case,1.0,0.960915,0.962321,0.942731,0.943950
4,4_Hour_Batteries_Case_Fuelx2,1.0,0.936296,0.939420,0.910085,0.912428
5,4_Hour_Batteries_Case_Fuelx3,1.0,0.917935,0.923195,0.888475,0.884348
6,4_Hour_Batteries_Case_Fuelx4,1.0,0.893777,0.898222,0.859198,0.851816
7,6_Hour_Batteries_Case,1.0,0.961139,0.962649,0.943690,0.946142
8,8_Hour_Batteries_Case,1.0,0.960461,0.962537,0.944890,0.947249
9,8_Hour_Batteries_Case_Fuelx2,1.0,0.937821,0.946312,0.913763,0.916593


In [ ]:
# calculate total system cost normalized by the CEM total system cost
total_system_costs_normalized = total_system_costs_df.copy()
# for col in total_system_costs_normalized.columns[1:]:
#     total_system_costs_normalized[col] = pd.to_numeric(total_system_costs_df[col]) / pd.to_numeric(total_system_costs_df['CEM'])
#     #OR pd.to_numeric(total_system_costs_df[col], errors='coerce') / pd.to_numeric(total_system_costs_df['CEM'], errors='coerce')
#     total_system_costs_normalized[col] = total_system_costs_normalized[col].round(1)

total_system_costs_normalized['CEM'] = total_system_costs_df['CEM'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['PF'] = total_system_costs_df['PF'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['DLAC'] = total_system_costs_df['DLAC'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['DLAC Imperfect'] = total_system_costs_df['DLAC Imperfect'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['SLAC'] = total_system_costs_df['SLAC'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0,-0.003209,-0.000363,-0.008053,-0.008962
1,2_Hour_Batteries_Case,0.0,-0.027221,-0.020981,-0.024961,-0.026730
2,2_Hour_Batteries_Case_Fuelx2,0.0,-0.034568,-0.029975,-0.033871,-0.036044
3,4_Hour_Batteries_Case,0.0,-0.026273,-0.021894,-0.023321,-0.024403
4,4_Hour_Batteries_Case_Fuelx2,0.0,-0.034850,-0.030535,-0.036042,-0.037151
5,4_Hour_Batteries_Case_Fuelx3,0.0,-0.040071,-0.037402,-0.044995,-0.047025
6,4_Hour_Batteries_Case_Fuelx4,0.0,-0.051016,-0.048974,-0.059008,-0.062071
7,6_Hour_Batteries_Case,0.0,-0.025482,-0.021645,-0.022039,-0.021842
8,8_Hour_Batteries_Case,0.0,-0.025066,-0.021675,-0.021970,-0.020004
9,8_Hour_Batteries_Case_Fuelx2,0.0,-0.030072,-0.026117,-0.033532,-0.032503


In [ ]:
total_system_costs_norm_percentage = total_system_costs_normalized.copy()
total_system_costs_norm_percentage['CEM'] = total_system_costs_normalized['CEM'] * 100
total_system_costs_norm_percentage['PF'] = total_system_costs_normalized['PF'] * 100
total_system_costs_norm_percentage['DLAC'] = total_system_costs_normalized['DLAC'] * 100
total_system_costs_norm_percentage['DLAC Imperfect'] = total_system_costs_normalized['DLAC Imperfect'] * 100
total_system_costs_norm_percentage['SLAC'] = total_system_costs_normalized['SLAC'] * 100

# Remove the decimal point from the normalized total system costs
total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']].round(2)
# Add a '%' symbol at the end of the normalized total system costs
total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

total_system_costs_norm_percentage .to_csv(tables_path + 'total_system_costs_norm_percentage .csv', index=False)
total_system_costs_norm_percentage 

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.0%,-0.320920,-0.04%,-0.81%,-0.9%
1,2_Hour_Batteries_Case,0.0%,-2.722091,-2.1%,-2.5%,-2.67%
2,2_Hour_Batteries_Case_Fuelx2,0.0%,-3.456760,-3.0%,-3.39%,-3.6%
3,4_Hour_Batteries_Case,0.0%,-2.627306,-2.19%,-2.33%,-2.44%
4,4_Hour_Batteries_Case_Fuelx2,0.0%,-3.484979,-3.05%,-3.6%,-3.72%
5,4_Hour_Batteries_Case_Fuelx3,0.0%,-4.007067,-3.74%,-4.5%,-4.7%
6,4_Hour_Batteries_Case_Fuelx4,0.0%,-5.101574,-4.9%,-5.9%,-6.21%
7,6_Hour_Batteries_Case,0.0%,-2.548209,-2.16%,-2.2%,-2.18%
8,8_Hour_Batteries_Case,0.0%,-2.506635,-2.17%,-2.2%,-2.0%
9,8_Hour_Batteries_Case_Fuelx2,0.0%,-3.007209,-2.61%,-3.35%,-3.25%


In [ ]:
# subtract total costs_df from consumer payments df to get profits
# Convert the columns to float for calculation
consumer_payments_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = consumer_payments_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(float)
total_system_costs_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = total_system_costs_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(float)

# Subtract total costs from consumer payments to get profits
mrktsurplus_df = consumer_payments_df.copy()
mrktsurplus_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = consumer_payments_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] - total_system_costs_df[['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']]

# Convert the profits to scientific notation
mrktsurplus_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = mrktsurplus_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].applymap(lambda x: f"{x:.2e}")

# Save the profits dataframe to a CSV file
mrktsurplus_df.to_csv(tables_path + 'mrktsurplus_df.csv', index=False)

mrktsurplus_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0.00e+00,-2.07e+08,1.01e+09,nan,6.67e+08
1,2_Hour_Batteries_Case,-1.77e+08,-1.71e+09,-1.45e+09,nan,-2.00e+09
2,2_Hour_Batteries_Case_Fuelx2,-3.04e+08,-2.51e+09,-2.73e+09,nan,-1.48e+09
3,4_Hour_Batteries_Case,-1.92e+08,-2.77e+09,-2.21e+09,nan,-2.16e+09
4,4_Hour_Batteries_Case_Fuelx2,-3.73e+08,-3.90e+09,-3.86e+09,nan,-3.47e+09
5,4_Hour_Batteries_Case_Fuelx3,-4.74e+08,-3.24e+09,-6.17e+09,nan,-4.06e+09
6,4_Hour_Batteries_Case_Fuelx4,-5.81e+08,-5.44e+09,-6.20e+09,nan,-4.45e+09
7,6_Hour_Batteries_Case,-1.98e+08,-2.64e+09,-1.94e+09,nan,-1.31e+09
8,8_Hour_Batteries_Case,-2.03e+08,-2.08e+09,-1.68e+09,nan,-3.20e+07
9,8_Hour_Batteries_Case_Fuelx2,-3.70e+08,-6.71e+09,-5.16e+09,nan,-4.96e+09


In [ ]:
# Create a new dataframe by dividing the 'DLAC', 'DLAC Imperfect', and 'SLAC' columns by the 'CEM' column
consumer_payments_ratio_df = consumer_payments_df.copy()
consumer_payments_ratio_df['CEM'] = consumer_payments_df['CEM'].astype(float) / consumer_payments_df['CEM'].astype(float) - 1
consumer_payments_ratio_df['PF'] = consumer_payments_df['PF'].astype(float) / consumer_payments_df['CEM'].astype(float) - 1
consumer_payments_ratio_df['DLAC'] = consumer_payments_df['DLAC'].astype(float) / consumer_payments_df['CEM'].astype(float) -1 
consumer_payments_ratio_df['DLAC Imperfect'] = consumer_payments_df['DLAC Imperfect'].astype(float) / consumer_payments_df['CEM'].astype(float) -1
consumer_payments_ratio_df['SLAC'] = consumer_payments_df['SLAC'].astype(float) / consumer_payments_df['CEM'].astype(float) -1

consumer_payments_ratio_df['CEM'] = consumer_payments_ratio_df['CEM'].round(2) * 100
consumer_payments_ratio_df['PF'] = consumer_payments_ratio_df['PF'].round(2) * 100
consumer_payments_ratio_df['DLAC'] = consumer_payments_ratio_df['DLAC'].round(2) * 100
consumer_payments_ratio_df['DLAC Imperfect'] = consumer_payments_ratio_df['DLAC Imperfect'].round(2) * 100
consumer_payments_ratio_df['SLAC'] = consumer_payments_ratio_df['SLAC'].round(2) * 100

# Remove the decimal point from the normalized total system costs
consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(int)

# Add a '%' symbol at the end of the normalized total system costs
consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

consumer_payments_ratio_df.to_csv(tables_path + 'consumer_payments_ratio.csv', index=False)
consumer_payments_ratio_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,0%,22433%,-1%,5%,3%
1,2_Hour_Batteries_Case,0%,-100%,-10%,-9%,-12%
2,2_Hour_Batteries_Case_Fuelx2,0%,-100%,-12%,-14%,-9%
3,4_Hour_Batteries_Case,0%,-100%,-15%,-12%,-12%
4,4_Hour_Batteries_Case_Fuelx2,0%,-100%,-18%,-18%,-17%
5,4_Hour_Batteries_Case_Fuelx3,0%,-100%,-14%,-26%,-18%
6,4_Hour_Batteries_Case_Fuelx4,0%,-100%,-22%,-26%,-20%
7,6_Hour_Batteries_Case,0%,-100%,-14%,-11%,-8%
8,8_Hour_Batteries_Case,0%,-100%,-12%,-10%,-1%
9,8_Hour_Batteries_Case_Fuelx2,0%,-100%,-28%,-23%,-22%


In [ ]:
total_operating_costs_df = pd.DataFrame.from_dict(total_operating_costs_dic, orient='index').reset_index()
total_operating_costs_df.columns = ['Case Name', 'PF', 'CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_operating_costs_df.to_csv(tables_path + 'total_operating_costs.csv', index=False)
total_operating_costs_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Case,9.6625e+09,9.6505e+09,9.6563e+09,9.4989e+09,9.4882e+09
1,2_Hour_Batteries_Case,1.0668e+10,1.0282e+10,1.0295e+10,1.0066e+10,1.0065e+10
2,2_Hour_Batteries_Case_Fuelx2,1.1778e+10,1.1077e+10,1.1091e+10,1.0762e+10,1.0776e+10
3,4_Hour_Batteries_Case,1.0669e+10,1.0252e+10,1.0267e+10,1.0058e+10,1.0071e+10
4,4_Hour_Batteries_Case_Fuelx2,1.1522e+10,1.0788e+10,1.0824e+10,1.0486e+10,1.0513e+10
5,4_Hour_Batteries_Case_Fuelx3,1.2356e+10,1.1342e+10,1.1407e+10,1.0978e+10,1.0927e+10
6,4_Hour_Batteries_Case_Fuelx4,1.3274e+10,1.1864e+10,1.1923e+10,1.1405e+10,1.1307e+10
7,6_Hour_Batteries_Case,1.0602e+10,1.0190e+10,1.0206e+10,1.0005e+10,1.0031e+10
8,8_Hour_Batteries_Case,1.0597e+10,1.0178e+10,1.0200e+10,1.0013e+10,1.0038e+10
9,8_Hour_Batteries_Case_Fuelx2,1.1306e+10,1.0603e+10,1.0699e+10,1.0331e+10,1.0363e+10


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# BEGIN: Add unit profit tables to PDF

# create a date stamp of just the year, month and day
date = datetime.datetime.now()
date = date.strftime("%Y-%m-%d-%H-%M")

In [ ]:
unit_profits_fln =  'unit_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + unit_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        unit_profit_df = unit_profit_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=unit_profit_df.values, colLabels=unit_profit_df.columns, cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'Unit Profits for {case_name} ($/MW)', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [ ]:
percentage_profits_fln =  'percentage_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + percentage_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        percentage_profits_df = percentage_profits_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=percentage_profits_df.values, \
                         colLabels=percentage_profits_df.columns, \
                            cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'% Profits over Fixed Costs for {case_name}', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [ ]:
# print 4 hour battery case percentage profits to csv
percentage_profits_dic['4_Hour_Batteries_Case'].to_csv(tables_path + '4_hour_battery_case_percentage_profits.csv', index=False)
percentage_profits_dic['4_Hour_Batteries_Case_Fuelx4'].to_csv(tables_path + '4_hour_battery_case_fuelx4_percentage_profits.csv', index=False)

In [ ]:
battery_df = pd.DataFrame()
for key in percentage_profits_dic:
    print(f"Key: {key}")
    # print(percentage_profits_dic[key])
    battery_row = percentage_profits_dic[key][percentage_profits_dic[key]['Resource'] == 'Battery']
    battery_row.insert(0, 'Case', key)
    battery_df = pd.concat([battery_df, battery_row], ignore_index=True)
    if battery_df.empty:
        battery_df = None
    # print(battery_df)

battery_df.to_csv(tables_path + 'battery_percentage_profits.csv', index=False)

Key: Thermal_Case
Key: 2_Hour_Batteries_Case
Key: 2_Hour_Batteries_Case_Fuelx2
Key: 4_Hour_Batteries_Case
Key: 4_Hour_Batteries_Case_Fuelx2
Key: 4_Hour_Batteries_Case_Fuelx3
Key: 4_Hour_Batteries_Case_Fuelx4
Key: 6_Hour_Batteries_Case
Key: 8_Hour_Batteries_Case
Key: 8_Hour_Batteries_Case_Fuelx2
Key: 10_Hour_Batteries_Case
Key: 10_Hour_Batteries_Case_Fuelx2


In [ ]:
# # Convert the percentage profits for 'Thermal_Case' to a DataFrame
# thermal_case_df = percentage_profits_dic['Thermal_Case']

# # Define the filename
# thermal_case_csv_filename = pdf_path + 'thermal_case_percentage_profits.csv'

# # Save the DataFrame to a CSV file
# thermal_case_df.to_csv(thermal_case_csv_filename, index=False)

In [ ]:
consumer_payments_fln =  'consumer_payments_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + consumer_payments_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=consumer_payments_df.values, colLabels=consumer_payments_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Consumer Payments for Each Case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

In [ ]:
consumer_payments_ratio_fln =  'consumer_payments_ratio_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + consumer_payments_ratio_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=consumer_payments_ratio_df.values, colLabels=consumer_payments_ratio_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Ratio of Consumer Payments (Normalized by CEM)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

In [ ]:
pmr_cpr_dic = {}

for case_name in case_names:
    percentage_profits_df = percentage_profits_dic[case_name]
    consumer_payments_ratio = consumer_payments_ratio_df[consumer_payments_ratio_df['Case Name'] == case_name].iloc[0]
    
    # Create a new row with the consumer payments ratio
    new_row = pd.DataFrame({
        'Resource': ['Consumer Payments Ratio'],
        'CEM': [f"{float(consumer_payments_ratio['CEM']) * 100:.2f}%"],
        'DLAC': [f"{float(consumer_payments_ratio['DLAC']) * 100:.2f}%"],
        'DLAC Imperfect': [f"{float(consumer_payments_ratio['DLAC Imperfect']) * 100:.2f}%"],
        'SLAC': [f"{float(consumer_payments_ratio['SLAC']) * 100:.2f}%"]
    })
    
    # Append the new row to the dataframe
    percentage_profits_df = pd.concat([percentage_profits_df, new_row], ignore_index=True)
    
    # Update the dictionary
    pmr_cpr_dic[case_name] = percentage_profits_df

# pmr_cpr_dic
# concatenate all the dataframes in pmr_cpr_dic, adding a row for the name of the case
pmr_cpr_combined_df = pd.DataFrame()

for case_name, df in pmr_cpr_dic.items():
    df.insert(0, 'Case Name', case_name)
    pmr_cpr_combined_df = pd.concat([pmr_cpr_combined_df, df], ignore_index=True)


# replace the names of the cases with the print names
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('2_Hour_Batteries_Case', '2 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('2_Hour_Batteries_Case_Fuelx2', '2 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case', '4 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx2', '4 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx3', '4 Hr Bat, Fuel Costs x3')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx4', '4 Hr Bat, Fuel Costs x4')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('6_Hour_Batteries_Case', '6 Hour Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('6_Hour_Batteries_Case_Fuelx2', '6 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('8_Hour_Batteries_Case', '8 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('8_Hour_Batteries_Case_Fuelx2', '8 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('10_Hour_Batteries_Case', '10 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('10_Hour_Batteries_Case_Fuelx2', '10 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('Thermal_Case', 'Thermal')

pmr_cpr_combined_df.to_csv(tables_path + 'pmr_cpr_combined_' + date + '.csv', index=False)

ValueError: could not convert string to float: '0%'

In [ ]:

total_system_costs_fln =  'total_system_costs_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_system_costs_fln) as pdf:

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_system_costs_df.values, colLabels=total_system_costs_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title(f'Total Costs for each case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)
# END: Add unit profit tables to PDF

In [ ]:
total_nse_df.columns

Index(['Case', 'CEM', 'DLAC', 'DLAC Imperfect', 'SLAC'], dtype='object')

In [ ]:
total_nse_fln =  'total_nse_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_nse_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_nse_df.values, colLabels=total_nse_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Non-served Energy for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

In [ ]:
total_unmet_rsv_fln =  'total_unmet_rsv_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_unmet_rsv_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_unmet_rsv_df.values, colLabels=total_unmet_rsv_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Unmet Reserves for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)